In [3]:
import pygrib
import numpy as np

In [7]:
global llat, ulat,llon,rlon
def CreateGrid(lon, lat, offside = 1,nx = 20, ny = 20):
    # offside is in the unit of degrees
    x = np.linspace(lon - offside, lon + offside, nx)
    y = np.linspace(lat - offside, lat + offside, ny)
    xv, yv = np.meshgrid(x, y, sparse = False)
    Grid2D = np.vstack((xv.flatten(),yv.flatten())).T
    # [Lon, Lat]
    return Grid2D

def GetIMG(lat, lon, lvl, Tree_Dict, WindClass, N_Pt = 20):
    Winds = WindClass.winds
    Grid = CreateGrid(lat, lon, offside = 1, nx = N_Pt, ny = N_Pt) # Lat, Lon
    try:
        QueryIdx1 = Tree_Dict[lvl].query(Grid)[1]  # base levels
        u_wind1 = Winds[0][0][WindClass.lvls[lvl]].data(llat, ulat, llon, rlon)[0][QueryIdx1]
        v_wind1 = Winds[0][1][WindClass.lvls[lvl]].data(llat, ulat, llon, rlon)[0][QueryIdx1]
        try:
            QueryIdx2 = Tree_Dict[lvl+25].query(Grid)[1]   # lower levels
            u_wind2 = Winds[0][0][WindClass.lvls[lvl+25]].data(llat, ulat, llon, rlon)[0][QueryIdx2]
            v_wind2 = Winds[0][1][WindClass.lvls[lvl+25]].data(llat, ulat, llon, rlon)[0][QueryIdx2]
        except:
            u_wind2 = u_wind1
            v_wind2 = v_wind1
        try:
            QueryIdx3 = Tree_Dict[lvl-25].query(Grid)[1]   # higher levels
            u_wind3 = Winds[0][0][WindClass.lvls[lvl-25]].data(llat, ulat, llon, rlon)[0][QueryIdx3]
            v_wind3 = Winds[0][1][WindClass.lvls[lvl-25]].data(llat, ulat, llon, rlon)[0][QueryIdx3]
        except:
            u_wind3 = u_wind1
            v_wind3 = v_wind1
    except:
        return 
    
    Wind_Cube = np.dstack((u_wind3.reshape(20,20), v_wind3.reshape(20,20), 
                           u_wind1.reshape(20,20), v_wind1.reshape(20,20), 
                           u_wind2.reshape(20,20), v_wind2.reshape(20,20)))
    return Wind_Cube

In [6]:
CreateGrid(-90, 25)

(400, 2)

In [48]:
st = time.time()
grbs = pygrib.open('../../filtered_weather_data/namanl/namanl_218_20130104_0000_003.grb')
uin = grbs.select(shortName='u', typeOfLevel='isobaricInhPa', level = lambda l: l >= 150 and l <= 1000)
vin = grbs.select(shortName='v', typeOfLevel='isobaricInhPa', level = lambda l: l >= 150 and l <= 1000)
# testU = grbs.select(shortName='v', typeOfLevel='pressureFromGroundLayer')
# for i in range(grbs.messages):
#     print(grbs.message(i+1))
grbs.close()
winds = [(uin, vin)]
print(time.time() - st)

0.6161456108093262


In [3]:
a = winds[0][0][0].data()

In [12]:
np.save('test.npy', a)

## Test load from zip

In [17]:
import zipfile
import io
import time
import os
import shutil

In [53]:
namelist[2][7:]

'namanl_218_20130101_0000_003.grb'

In [6]:
zwind = zipfile.ZipFile('../../filtered_weather_data/namanl/namanl.zip', mode = 'r')
namelist = zwind.namelist()

In [64]:
tmp = io.BytesIO(zwind.open(name = namelist[1]).read())
tmp2 = io.StringIO(namelist[1])

tmp_bytes = zwind.open(name = namelist[1]).read()

In [ ]:
grbs = pygrib.open('../../filtered_weather_data/namanl/namanl.zip/')

In [16]:
len(tmp_bytes)

31347210

In [60]:
st = time.time()
zwind.extract(member=namelist[50], path='../weather/namanl')
shutil.rmtree(path = '../weather/namanl/')
print(time.time() - st)

0.291705846786499


In [ ]:
zwind.close()